In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

import glob
import os

%matplotlib inline

In [2]:
cwd = os.getcwd()

SAMPLE_SIZE = 5
SPLIT_RATIO = 0.7

PADDING_SIZE = 5
IMAGE_SIZE = 256
BATCH_SIZE = 16

In [3]:
index = np.arange(SAMPLE_SIZE) + 1
np.random.shuffle(index)

split = (int) (SAMPLE_SIZE * SPLIT_RATIO)
x_train_idx = index[0:split]
x_test_idx = index[split:]

In [4]:
ALL_WORD = ['', 'statement', 'if', 'else', 'elseif', 'for', 'while', 'end']
N_ONEHOT_WORD = len(ALL_WORD) - 1
CHARS_MAP = {v: k for k, v in enumerate(ALL_WORD)}
IDX_MAP = dict(list(enumerate(ALL_WORD)))

def to_onehot(word):
    n_onehot = N_ONEHOT_WORD
    idx = CHARS_MAP[word]
    if idx == 0:
        return np.zeros(n_onehot)
    else:
        onehot_vec = np.zeros(n_onehot)
        onehot_vec[idx - 1] = 1
        return onehot_vec

def convert_to_feature_list(feature_words):
    onehots = []
    for word in feature_words:
        onehots.append(to_onehot(word))
    return np.array(onehots)
        

def convert_to_input_set(tokens, k):
    n_tokens = len(tokens)
    padded_token = [''] * k + tokens
    res = []
    for i in range(n_tokens - 1):
        res.append((
            convert_to_feature_list(padded_token[i:i + k]),
            to_onehot(padded_token[i + k])
        ))
    return res

In [5]:
def input_generator(indexs):
    while(True):
        x_word = []
        x_image = []
        y = []
        
        for idx in indexs:
            picture_files = glob.glob("./data/sample-" + str(idx) + "-*.jpg")
            lang_file = glob.glob('./data/sample-' + str(idx) + '-*.txt')
            
            if len(lang_file) == 0:
                continue
            
            with open(lang_file[0], 'r') as file:
                lang = [l.strip().split(" ")[0] for l in file.read().split("\n") if len(l)]
                lang = list(map(lambda x: "statement" if x.startswith("statement") else x, lang))
            
            embed_lang = convert_to_input_set(lang, PADDING_SIZE)
            
            for picture in picture_files:
                image = Image.open(picture).convert('L')
                image = image.convert()
                image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
                image = np.asarray(image).reshape((IMAGE_SIZE, IMAGE_SIZE, 1))
                image = image / 255

                for lang in embed_lang:
                    x_word.append(lang[0])
                    x_image.append(image)
                    y.append(lang[1])
                    
                    if len(y) >= BATCH_SIZE:
                        yield [np.array(x_word), np.array(x_image)], np.array(y)
                        x_word = []
                        x_image = []
                        y = []

In [6]:
from keras.layers import Input, Dense, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout, LSTM
from keras.models import Model

def generate_model():
    word_input = Input(shape=(PADDING_SIZE, N_ONEHOT_WORD))
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(64, (7, 7), padding='same', activation='relu')(image_input)
    img = Conv2D(64, (7, 7), padding='same', activation='relu')(image_input)
    img = MaxPooling2D()(img)
    img = Conv2D(128, (5, 5), padding='same', activation='relu')(img)
    img = Conv2D(128, (5, 5), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Conv2D(256, (5, 5), padding='same', activation='relu')(img)
    img = Conv2D(256, (5, 5), padding='same', activation='relu')(img)
    img = Conv2D(256, (5, 5), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Conv2D(256, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(256, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(256, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Conv2D(512, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(512, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(512, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Flatten()(img)
    img = Dense(1024, activation='relu')(img)
    
    w = LSTM(32, return_sequences=True)(word_input)
    w = LSTM(32, return_sequences=True)(w)
    w = LSTM(32)(w)
    
    x = Concatenate()([w, img])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    outputs = Dense(N_ONEHOT_WORD, activation='softmax')(x)

    model = Model(inputs=[word_input, image_input], outputs=outputs)
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

print(generate_model().summary())

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 3200        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 128, 128, 128 204928      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_4 (

In [7]:
from keras.layers import Input, Dense, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout, LSTM
from keras.models import Model

def generate_model_small():
    word_input = Input(shape=(PADDING_SIZE, N_ONEHOT_WORD))
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(4, (3, 3), padding='same', activation='relu')(image_input)
    img = MaxPooling2D()(img)
    img = Conv2D(8, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D()(img)
    img = MaxPooling2D()(img)
    img = MaxPooling2D()(img)
    img = Dropout(0.1)(img)
    img = Flatten()(img)
    img = Dense(16, activation='relu')(img)
    
    w = LSTM(8, return_sequences=True)(word_input)
    w = LSTM(8)(w)
    
    x = Concatenate()([w, img])
    x = Dense(8, activation='relu')(x)
    
    outputs = Dense(N_ONEHOT_WORD, activation='softmax')(x)

    model = Model(inputs=[word_input, image_input], outputs=outputs)
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

print(generate_model_small().summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 256, 256, 4)  40          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 128, 128, 4)  0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 128, 8)  296         max_pooling2d_6[0][0]            
__________________________________________________________________________________________________
max_poolin

In [8]:
from keras.callbacks import Callback
from keras import backend as K

class TensorBoard(Callback):

    def __init__(self, log_dir='./logs',
                 write_graph=False,
                 start_steps=0,
                 batch_freq=1):
        super(TensorBoard, self).__init__()
        
        global tf, projector
        import tensorflow as tf
        from tensorflow.contrib.tensorboard.plugins import projector
        
        self.log_dir = log_dir
        self.batch_freq = batch_freq
        self.write_graph = write_graph
        
        self.start_steps = start_steps
        self.steps_counter = 1

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()
        
        self.merged = tf.summary.merge_all()

        if self.write_graph:
            self.writer = tf.summary.FileWriter(self.log_dir,
                                                self.sess.graph)
        else:
            self.writer = tf.summary.FileWriter(self.log_dir)
            
    def save_scalar(self, logs):
        log = logs or {}
        
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(
                summary,
                self.start_steps + self.steps_counter
            )
        self.writer.flush()
        
        self.steps_counter += 1
            
    def on_batch_end(self, batch, logs=None):
        if self.steps_counter % self.batch_freq == 0:
            self.save_scalar(logs)

    def on_epoch_end(self, epoch, logs=None):
        self.save_scalar(logs)        

    def on_train_end(self, _):
        self.writer.close()

In [9]:
model = generate_model_small()

model.fit_generator(
    input_generator(x_train_idx),
    steps_per_epoch=20,
    validation_data = input_generator(x_test_idx),
    validation_steps = 10,
    max_queue_size=5,
    epochs=3,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(
            log_dir="./model/logs/",
            batch_freq=5
        )
    ]
)

Epoch 1/3
20/20 [==============================] - 10s 495ms/step - loss: 1.6086 - acc: 0.5781 - val_loss: 1.4894 - val_acc: 0.6000
Epoch 2/3
20/20 [==============================] - 9s 471ms/step - loss: 1.4669 - acc: 0.5844 - val_loss: 1.4060 - val_acc: 0.6000
Epoch 3/3
20/20 [==============================] - 10s 497ms/step - loss: 1.4796 - acc: 0.5656 - val_loss: 1.3850 - val_acc: 0.6000
